# 06-05: vLLM Offline Inference

In [1]:
import mlrun
from IPython.display import Markdown, display

## 1. Configuration

In [2]:
MODEL_NAME = "Mistral-7B-Instruct-v0.2-AWQ"
project_name = "test-vllm-integration" # the project name

### 1.1 Load The Project

In [3]:
project = mlrun.get_or_create_project(
    name=project_name,
    user_project=False)

# Display the current project name
project_name = project.metadata.name
print(f'Full project name: {project_name}')

> 2025-08-12 12:02:50,162 [info] Project loaded successfully: {"project_name":"test-vllm-integration"}
Full project name: test-vllm-integration
